# Importations

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install rake-nltk

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)
import pandas as pd
import numpy as np
from rake_nltk import Rake
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

# Tri du DF

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/PARIS 0224 - Data Analyst/Datathon/Restolympic/Datasets/Dataset - Pré nettoyage/Copie de tripadvisor_restos.csv",sep=';')

In [ ]:
print(df.head().to_markdown())

|    | restaurant_link   | restaurant_name   | city   | address                                                 |   latitude |   longitude | top_tags                                 | price_level   | meals                                  | cuisines                   |   special_diets | features                                                                                                                                                                                   | vegetarian_friendly   | vegan_options   | gluten_free   | original_open_hours                                                                                                                                                      |   avg_rating |   total_reviews_count |   excellent |   very_good |   average |   poor |   terrible |   food |   service |   value |
|---:|:------------------|:------------------|:-------|:--------------------------------------------------------|-----------:|------------:|:---------------------

In [ ]:
df["top_tags"] = df.top_tags.str.replace(" ","").str.replace(",", " ")

In [ ]:
df.columns

Index(['restaurant_link', 'restaurant_name', 'city', 'address', 'latitude',
       'longitude', 'top_tags', 'price_level', 'meals', 'cuisines',
       'special_diets', 'features', 'vegetarian_friendly', 'vegan_options',
       'gluten_free', 'original_open_hours', 'avg_rating',
       'total_reviews_count', 'excellent', 'very_good', 'average', 'poor',
       'terrible', 'food', 'service', 'value'],
      dtype='object')

In [ ]:
df  = df.drop(['restaurant_link','city','excellent', 'very_good', 'average', 'poor',
       'terrible', 'food', 'service', 'value'], axis=1)

In [ ]:
df = df.drop(['vegetarian_friendly', 'vegan_options','gluten_free'], axis=1)

In [ ]:
print(df.head(30).to_markdown())

|    | restaurant_name                  | address                                                                                  |   latitude |   longitude | top_tags                                 | price_level   | meals                                  | cuisines                                  | special_diets       | features                                                                                                                                                                                   | original_open_hours                                                                                                                                                                                                                                                               |   avg_rating |   total_reviews_count |
|---:|:---------------------------------|:-----------------------------------------------------------------------------------------|-----------:|------------:|:----------

In [ ]:
print(df.shape)

(1708, 13)


In [ ]:
df = df.fillna("")

In [ ]:
df['mots'] = df['top_tags'] + ' '+ df['meals'] + " " +  df['special_diets'] + " " + df['features']

In [ ]:
print(df.head().to_markdown())

|    | restaurant_name   | address                                                 |   latitude |   longitude | top_tags                            | price_level   | meals                                  | cuisines                   | special_diets   | features                                                                                                                                                                                   | original_open_hours                                                                                                                                                      |   avg_rating |   total_reviews_count | mots                                                                                                                                                                                                                                                |
|---:|:------------------|:--------------------------------------------------------|-----------:|----

# Modele ML

#### choisir  les tag de cuisine

In [ ]:
cuisines = df.top_tags + " " + df.cuisines
print(cuisines.str.split(" ").explode().unique())

['FineDining' 'Italian' 'European' 'CheapEats' 'Pizza' 'Belgian'
 'Fastfood' 'Fast' 'food' 'Mid-range' 'Mediterranean' 'French' 'Seafood'
 'Healthy' 'QuickBites' 'Cafe' '' 'Chinese' 'Asian' 'Thai' 'Vietnamese'
 'Lebanese' 'Bar' 'Bakeries' 'American' 'Pub' 'Canadian' 'Gastropub'
 'Dessert' 'Street' 'Food' 'Soups' 'Japanese' 'Sushi' 'Basque'
 'VegetarianFriendly' 'Korean' 'Spanish' 'Turkish' 'MiddleEastern'
 'Middle' 'Eastern' 'GlutenFreeOptions' 'Deli' 'Kosher' 'BrewPub' 'Brew'
 'Steakhouse' 'Central' 'International' 'Greek' 'VeganOptions' 'Grill'
 'StreetFood' 'Peruvian' 'Moroccan' 'Indian' 'Barbecue' 'African'
 'JapaneseFusion' 'Fusion' 'SpecialityFoodMarket' 'Portuguese' 'WineBar'
 'Wine' 'Romana' 'Lazio' 'Central-Italian' 'Diningbars' 'Dining' 'bars'
 'Brazilian' 'Ethiopian' 'South' 'Arabic' 'Neapolitan' 'Campania'
 'Sicilian' 'Southern-Italian' 'Cambodian' 'Tunisian' 'Mexican' 'Swedish'
 'Scandinavian' 'Russian' 'SouthAmerican' 'Contemporary' 'Diner'
 'Pakistani' 'Bangladeshi' 'Alg

In [ ]:
c = 'Tunisian Street Food Pizza '


#### choisir  le timing du repas

In [ ]:
print(df.meals.str.split(" ").explode().unique())


['Lunch' 'Dinner' '' 'Brunch' 'After-hours' 'Drinks' 'Breakfast']


In [ ]:
periode = "Lunch"

#### choisir autres features

In [ ]:

print(df.features.str.split(" ").explode().unique())

a = "Music"

['Reservations' 'Seating' 'Wheelchair' 'Accessible' 'Serves' 'Alcohol'
 'Table' 'Service' 'Television' 'Wine' 'and' 'Beer' 'Accepts' 'Mastercard'
 'Visa' 'Digital' 'Payments' 'Free' 'Wifi' 'Takeout' 'Outdoor' 'Credit'
 'Cards' 'Full' 'Bar' 'Highchairs' 'Available' 'Delivery' 'American'
 'Express' 'Discover' 'Private' 'Dining' 'Street' 'Parking' 'Live' 'Music'
 'Gift' 'Buffet' 'Family' 'style' 'Non-smoking' 'restaurants' 'Valet'
 'Waterfront' 'Beach' 'Dog' 'Friendly' 'Drive' 'Thru' 'Validated'
 'off-street' 'parking' 'Cash' 'Only' 'Jazz' 'Sports' 'bars']


#### mots choisit par client

In [ ]:
all  = periode + " " + a + " " + c
print(all)
all_serie = pd.Series([all])
print(all_serie)

Lunch Music Tunisian Street Food Pizza 
0    Lunch Music Tunisian Street Food Pizza 
dtype: object


#### encodage et similarity

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
#cv = CountVectorizer(stop_words='english',token_pattern=r"(?u)\b[a-zA-Z0_9]+\b")
cv = TfidfVectorizer(stop_words='english',token_pattern=r"(?u)\b[a-zA-Z0_9]+\b")

In [ ]:
vectors=cv.fit_transform(df['mots']).toarray()

In [ ]:
new = cv.transform(all_serie).toarray()

In [ ]:
similarity=cosine_similarity(vectors,new)
sim_serie = pd.DataFrame(similarity,columns = ['reco']).sort_values("reco", ascending = False)
print(sim_serie)

          reco
788   0.473226
296   0.442975
968   0.284831
1645  0.277431
1567  0.276480
...        ...
1487  0.000000
733   0.000000
1167  0.000000
1170  0.000000
1302  0.000000

[1708 rows x 1 columns]


In [ ]:
reco_indice = sim_serie.head().index
final = df.iloc[reco_indice]
print(final.to_markdown())

|      | restaurant_name             | address                                        |   latitude |   longitude | top_tags                         | price_level   | meals                         | cuisines               | special_diets   | features                                                                                                             | original_open_hours                                                                                                                                                                                                                    |   avg_rating |   total_reviews_count | mots                                                                                                                                                                |
|-----:|:----------------------------|:-----------------------------------------------|-----------:|------------:|:---------------------------------|:--------------|:------------------------------|:----